In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from sklearn.neighbors import KDTree
import time
import argparse
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile
from skimage.color import rgb2hsv,rgb2lab, hsv2rgb, lab2rgb
import os
import matplotlib.pyplot as plt

In [2]:
# read the .xml-file
zip_ref = zipfile.ZipFile('../CompanyMen_v1.0-split-012-Bobby_being_angry.azp')
zip_ref.extractall('/tmp')
tree = ET.parse('/tmp/content.xml')
root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
i=0
ts_end=[]
ts_begin=[]
for child in root[0].iter():
    if child.get('type')=='#Shot':
        i+=1
        for child2 in child:
            if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                ts_end.append(round(int(child2.get('end'))/1000*25))
                ts_begin.append(round(int(child2.get('begin'))/1000*25))

In [3]:
print('ts_begin: ',np.shape(ts_begin),ts_begin)
print('ts_end: ',np.shape(ts_end),ts_end)

ts_begin:  (50,) [33020, 33112, 33242, 33268, 33320, 33386, 33428, 33532, 33572, 33714, 33768, 33806, 33864, 33922, 34006, 34056, 34116, 34154, 34190, 34248, 34408, 34452, 34504, 34692, 34906, 35070, 35150, 35280, 35746, 36126, 36256, 36302, 36466, 36524, 36558, 36616, 36798, 36856, 36916, 36956, 37008, 37036, 37074, 37112, 37134, 37186, 37256, 37286, 37312, 37390]
ts_end:  (50,) [33112, 33242, 33268, 33320, 33386, 33428, 33532, 33572, 33714, 33768, 33806, 33864, 33922, 34006, 34056, 34116, 34154, 34190, 34248, 34408, 34452, 34504, 34692, 34906, 35070, 35150, 35280, 35746, 36126, 36256, 36302, 36466, 36524, 36558, 36616, 36798, 36856, 36916, 36956, 37008, 37036, 37074, 37112, 37134, 37186, 37256, 37286, 37312, 37390, 37556]


In [4]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width,target_colorspace):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max lenght of the given segment
        while(vid.isOpened()):
            # Capture frame-by-frame
            ret, frame = vid.read() # if ret is false, frame has no content
            if not ret:
                break
            # skip every "skip_frame"
            if vid_length>=start_frame:
                # resize the video to a different resolution
                frame=cv2.resize(frame,resolution)
                frame=np.array(frame,dtype='uint8')
                frames.append(frame) #add the individual frames to a list
                pbar.update(1) #update the progressbar
            if vid_length==end_frame:
                pbar.update(1)
                break
            vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    frames=change_colorspace(frames,target_colorspace)
    return frames[:-1]

In [5]:
def change_colorspace(frame_list,target_colorspace):
    changed_frame_list=[]
    if target_colorspace=='HSV':
        print('HSV')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2HSV))
        return changed_frame_list
    if target_colorspace=='cie-lab':
        print('cie-lab')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2LAB))
        return changed_frame_list
    else:
        print('rgb')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        return changed_frame_list

In [6]:
def fn_rgb_to_color(target_colorspace,path):
            if (path != 'full'):
                colors = {}
                with open(path) as f:
                    for line in f:
                        #split lines at "::
                        color, rgb = line.strip().split(':')
                        #strip the rgb-string of the parenthesis, split it up a the commas,
                        #cast them to int and put them into a tuples
                        rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                        colors[color]=rgb_value
            else:
                colors={'darkred':(139,0,0),
                'firebrick':(178,34,34),
                'crimson':(220,20,60),
                'red':(255,0,0),
                'tomato':(255,99,71),
                'salmon':(250,128,114),
                'darkorange':(255,140,0),
                'gold':(255,215,0),
                'darkkhaki':(189,183,107),
                'yellow':(255,255,0),
                'darkolivegreen':(85,107,47),
                'olivedrab':(107,142,35),
                'greenyellow':(173,255,47),
                'darkgreen':(0,100,0),
                'aquamarine':(127,255,212),
                'steelblue':(70,130,180),
                'skyblue':(135,206,235),
                'darkblue':(0,0,139),
                'blue':(0,0,255),
                'royalblue':(65,105,225),
                'purple':(128,0,128),
                'violet':(238,130,238),
                'deeppink':(255,20,147),
                'pink':(255,192,203),
                'antiquewhite':(250,235,215),
                'saddlebrown':(139,69,19),
                'sandybrown':(244,164,96),
                'ivory':(255,255,240),
                'dimgrey':(105,105,105),
                'grey':(28,128,128),
                'silver':(192,192,192),
                'lightgrey':(211,211,211),
                'black':(0,0,0),
                'white':(255,255,255),
                'darkcyan':(0,139,139),
                'cyan':(0,255,255),
                'green':(0,128,0),
                'khaki':(240,230,140),
                'goldenrod':(218,165,32),
                'orange':(255,165,0),
                'coral':(255,127,80),
                'magenta':(255,0,255),
                'wheat':(245,222,179),
                'skin':(255,224,189),
                'purple4':(147,112,219)}

            colors_aux={}
            if target_colorspace=='HSV':
                print('HSV')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2HSV)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux
            if target_colorspace=='cie-lab':
                print('cie-lab')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2LAB)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux

            rgb_to_color={}
            for color in colors:
                rgb_to_color[colors[color]]=color
            #purple4 is median purple
            #skin is caucasian        
            return rgb_to_color

In [7]:
#params
resolution=200

In [8]:
scene_all_frames_lab=[]
for i,ts in enumerate(zip(ts_begin,ts_end)):
    scene_all_frames_lab.append(read_video_segments('../../Wells_John_CompanyMen_full.mp4'
                                                    ,ts[0],ts[1],resolution,'cie-lab'))
    if i==5:
        break

94it [00:40,  2.32it/s]                         
  0%|          | 0/131 [00:00<?, ?it/s]

cie-lab


132it [00:40,  3.26it/s]                         
  0%|          | 0/27 [00:00<?, ?it/s]

cie-lab


28it [00:42,  1.51s/it]                       
  0%|          | 0/53 [00:00<?, ?it/s]

...

cie-lab


68it [00:41,  1.65it/s]                        
  0%|          | 0/43 [00:00<?, ?it/s]

cie-lab


44it [00:43,  1.02it/s]                       

cie-lab


In [10]:
print('shape: ', np.shape(scene_all_frames_lab))

shape:  (6,)


In [11]:
def extract_dominant_colors(frame_list,target_colorspace,path):
    print(str(len(frame_list))+' frames to process.')
    rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
    bins={} #bins dict for histograms 
    for rgb in rgb_to_color: #init the dict with ones for every key to avoid difficulties with divisions
                            # because the the sum of the bins goes from 500k to 2kk this shouldn't be a problem
        bins[rgb_to_color[rgb]]=1
    rgb_list=[] #create a traverseable list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)

    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')
    for image in tqdm(frame_list): #traverse the video
        img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
        nns = kdt.query(img, k=1, return_distance=False)
        for nn in nns:
            bins[rgb_to_color[rgb_list[nn[0]]]]+=1
    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1] #normalize the bins
    bins_norm={k:v/norm_factor*10000 for k,v in bins.items()} #scale 0-10000 for visibility
    return bins_norm

In [12]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe
    bins_sorted=list(zip(list(bins.values()),list(bins.keys())))
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
#     bin_threshold=bin_threshold/100 #scale the percentage to 0-1
#     df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df

In [13]:
dataframes=[]
for shot in scene_all_frames_lab:
    bins=extract_dominant_colors(shot,'cie-lab','full')
    dataframes.append(bins_to_df(bins))

  8%|▊         | 7/92 [00:00<00:01, 62.75it/s]

92 frames to process.
cie-lab


  5%|▌         | 7/130 [00:00<00:01, 66.51it/s]

130 frames to process.
cie-lab


 23%|██▎       | 6/26 [00:00<00:00, 56.40it/s]

...

 11%|█         | 7/66 [00:00<00:00, 66.15it/s]

66 frames to process.
cie-lab


 17%|█▋        | 7/42 [00:00<00:00, 65.32it/s]

42 frames to process.
cie-lab


100%|██████████| 42/42 [00:00<00:00, 64.90it/s]


In [118]:
def process_df(dataframe_list,noise_threshold):
    # create the lists that are returned
    # add the first entry of the histogramm to the output 
    # the 'real' output is the real_dataframe_list
    absolute_dataframe_list=[dataframe_list[0].sort_values(by='count',ascending=False)+1]
    relative_dataframe_list=[dataframe_list[0].sort_values(by='count',ascending=False)+1]
    real_dataframe_list=[dataframe_list[0].sort_values(by='count',ascending=False)+1]
    
    # traverse the histograms
    # d is the current shot, d1 the following shot
    for d,d1 in zip(dataframe_list,dataframe_list[1:]):
        
        absolute_df=d1-d #calculate the absolute change of the histograms
        #add the changes to the corresponding lists
        absolute_dataframe_list.append(absolute_df)

        #apply a noise-filter to the absolute df
        absolute_high=absolute_df[absolute_df>noise_threshold].dropna()
        absolute_low=absolute_df[absolute_df<-noise_threshold].dropna()
        absolute_denoised=absolute_high.combine_first(absolute_low)
        
        # create the relative change, scaled to 0-100 percent
        # they are calculated from the denoised df, to prevent errors because of the noise
        relative_df=absolute_denoised/d*100
        relative_df=relative_df.dropna()
        relative_dataframe_list.append(relative_df)
        
        # if there are no elements in the denoised df it is assumed that all changes from
        # shot to shot are noise, in that case the current histogram is appended,
        # else the relative histogram
        if len(absolute_denoised)==0:
            real_dataframe_list.append(d.sort_values(by='count',ascending=False))
        else:
            real_dataframe_list.append(relative_df.sort_values(by='count',ascending=False))
            
#         print('len absolute_denoised: ',len(absolute_denoised))
        
    return {'real': real_dataframe_list, 'absolute':absolute_dataframe_list,'relative':relative_dataframe_list}

In [111]:
new = process_df(dataframes,100)

In [117]:
new['real'][1].head()

,count
color,
darkkhaki,164.212143
saddlebrown,54.742283
black,10.629691
dimgrey,-45.838163


In [126]:
for i,frame_list in enumerate(scene_all_frames_lab):
    cv2.imwrite('key_frames/'+str(i)+'.png',cv2.cvtColor(frame_list[0], cv2.COLOR_LAB2BGR))

In [116]:
a=pd.DataFrame([10,11,9,10,10,60])
b=pd.DataFrame([9,12,6,14,20,80])
c=pd.DataFrame([21,6,12,14,24,70])
abc=[a,b,c]
abs_abc=[a]
rel_abc=[a]
for d,d1 in zip(abc,abc[1:]):
#     print(d)
    abs_abc.append(d1-d)
    rel_abc.append((d1-d)/d*100)

In [118]:
# abs(abs_abc[1])/a*100

In [119]:
abs_abc[1]

,0
0,-1
1,1
2,-3
3,4
4,10
5,20


In [120]:
abs_abc[2]

,0
0,12
1,-6
2,6
3,0
4,4
5,-10


In [121]:
rel_abc[1]

,0
0,-10.000000
1,9.090909
2,-33.333333
3,40.000000
4,100.000000
5,33.333333


In [122]:
rel_abc[2]

,0
0,133.333333
1,-50.000000
2,100.000000
3,0.000000
4,20.000000
5,-12.500000
